In [7]:
import pandas as pd
import tensorflow as tf
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys
from tabulate import tabulate

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
                
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

rootpath=os.path.expanduser('~/tmp/logs/motion')
tags=['train/acc','val/acc']

['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [8]:
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['/stn_u','wo_stn_u']
note_gtags=[['upsample_layer','freeze_layer'],
            ['upsample_layer','freeze_layer']]
describe=[]
for note,ntag in zip(notes,note_gtags):
    show_tags=[]
    show_tags=tags.copy()
    group_tags=['note']+ntag
    show_tags+=group_tags
    show_tags+=['epoch']
    tasks=summary(rootpath,tags,note)
    if tasks.empty:
        print(note,ntag,'task is empty')
        continue
        
    #print(tasks[show_tags].groupby(group_tags).max().to_string())
    print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values('val/acc'),tablefmt='pipe',headers='keys'))
    print('\n')
    print(tabulate(tasks[['val/acc']+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
    print('\n')
    print(tabulate(tasks[show_tags].sort_values('val/acc'),tablefmt='pipe',headers='keys'))
    print('\n')

|                                 |   train/acc |   val/acc |   epoch |
|:--------------------------------|------------:|----------:|--------:|
| ('stn_u3f{freeze_layer}', 3, 3) |    0.916383 |  0.854048 |      30 |
| ('stn_u5f{freeze_layer}', 5, 3) |    0.959955 |  0.854574 |      30 |
| ('stn_u3f{freeze_layer}', 3, 0) |    0.987225 |  0.860692 |      30 |
| ('stn_u4f{freeze_layer}', 4, 3) |    0.966873 |  0.863473 |      30 |
| ('stn_u3f{freeze_layer}', 3, 2) |    0.967766 |  0.863873 |      30 |
| ('stn_u4f{freeze_layer}', 4, 2) |    0.980876 |  0.865827 |      30 |
| ('stn_u4f{freeze_layer}', 4, 1) |    0.985273 |  0.869544 |      30 |
| ('stn_u3f{freeze_layer}', 3, 1) |    0.985943 |  0.869708 |      30 |
| ('stn_u5f{freeze_layer}', 5, 2) |    0.977819 |  0.869732 |      30 |
| ('stn_u4f1', 4, 1)              |    0.986909 |  0.875642 |      30 |
| ('stn_u5f{freeze_layer}', 5, 1) |    0.987404 |  0.880334 |      30 |
| ('stn_u4f{freeze_layer}', 4, 0) |    0.987871 |  0.88127  |   